In [1]:
import os
import json
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
import library.validation.EnergyData.parsing as parsing

In [2]:
ticket_path_nep = "/home/lhn3e/OneDrive-3E/Research/Solar/tickets/2023/IN2946_EnergyData_measurement_data"
raw_path_nep =  os.path.join(ticket_path_nep, 'raw_data')
intermediate_path_nep =  os.path.join(ticket_path_nep, 'intermediate_data')

In [3]:
with open(os.path.join(ticket_path_nep,'metadata/data.json'), 'r') as file:
    stations_metadata_json = file.read()

stations_metadata = json.loads(stations_metadata_json)


In [4]:
station_name_list = []
for station in stations_metadata:
    station_name = f"{station['station name'].strip()}"
    station_name_list.append(station_name)


In [6]:
files = os.listdir(raw_path_nep)

file_list_sk = ['bangladesh', 'vietnam', 'lebanon']
file_list_en = ['benin', 'burkinafaso', 'cotedivoire', 'gambia', 'ghana', 'guinea', 'liberia', 'mali', 'nigeria', 'niger', 'measurements_senegal', 'sierraleone', 'togo']
file_list_flag = ['BDFE2 (Feni).csv', 'VNCEH (Central Highlands).csv', 'VNHAN (Hanoi).csv', 'VNTRA (Tri An).csv']

i = 0
dfs = {}

for file in files:
    data_file_path = os.path.join(raw_path_nep, file)
    print(f'saving {file}')
    
    # Read
    
    if any (fi in file for fi in file_list_sk):
        df = pd.read_csv(data_file_path, skiprows = 1)
        
    elif any (fi in file for fi in file_list_en):
        df = pd.read_csv(data_file_path, encoding='cp1252', skiprows=[1])
    
    elif file.startswith('solar-measurementssenegal-'):
        df = pd.read_csv(data_file_path, sep = ';')
        
    else:
        df = pd.read_csv(data_file_path)    
        
        
    # Concat
    
    if 'year' in file:
        place = file.split('_')[1]
        key = f'{place}'

        if key in dfs:
            df_c = pd.concat([dfs[key], df], ignore_index=True)

        else:
            dfs[key] = df.copy()
            df_c = None
            
    else:
        df_c = df.copy()
            
        
    # Parse    
    
    if df_c is not None:
        df_p = df_c.copy()
        df_p.columns = df_p.columns.str.upper()
        
        if '10min' in file:
            df_p = df_p.rename(columns={'#YEAR': 'YEAR'})
            DateTime = df_p[['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE']]
            df_p = parsing.parse(data_file_path, file, df_p, DateTime, '10T')
        
        elif 'pakistan' in file:
            DateTime = df_p[df_p.filter(like='TIME').columns[0]]
            df_p = parsing.parse(data_file_path, file, df_p, DateTime, '10T')
        
        else:
            DateTime = df_p[df_p.filter(like='TIME').columns[0]]
            df_p = parsing.parse(data_file_path, file, df_p, DateTime)
            
    if file in file_list_flag:
        df_p.drop(df_p.columns[-1], axis=1, inplace=True)


    # Save
    
        file_name = f"{station_name_list[i]}.csv"
        i = i+1
        
        file_path = os.path.join(intermediate_path_nep, file_name)
        df_p.to_csv(file_path, index_label='DateTime')

        print(f"Saved {file_name}")
    
    

saving hrazdanwbarmenia10min.csv
Saved Hrazdan.csv
saving masrikwbarmenia10min.csv
Saved Masrik.csv
saving solar-measurementspakistanhyderabadwb-esmapqc.csv
Saved PK_Solar_Hyderabad_MUET.csv
saving solar-measurementspakistanislamabadwb-esmapqc.csv
Saved PK_Solar_Islamabad_NUST.csv
saving solar-measurementspakistankarachiwb-esmapqc.csv
Saved PK_Solar_Karachi_NEDUET.csv
saving solar-measurementspakistankhuzdarwb-esmapqc.csv
Saved PK_Solar_Khuzdar_BUET.csv
saving solar-measurementspakistanlahorewb-esmapqc.csv
Saved PK_Solar_Lahore_UET.csv
saving solar-measurementspakistanmultanwb-esmapqc.csv
Saved PK_Solar_Multan_MSNUET.csv
saving solar-measurementspakistanpeshawarwb-esmapqc.csv
Saved PK_Solar_Peshawar_UET.csv
saving solar-measurementspakistanquettawb-esmapqc.csv
Saved PK_Solar_Quetta_BUITEMS.csv
saving solar-measurementssenegal-fatickifcqc.csv
Saved SN_Solar_Fatick_IFC.csv
saving solar-measurementssenegal-kahoneifcqc.csv
Saved SN_Solar_Kahone_IFC.csv
saving solar-measurementssenegal-toub